# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [53]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [54]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [55]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("/Users/domenicrhedrick/Desktop/Presentations/Pythonic Monopoly/neighborhoods_coordinates.csv")
neighborhood_data = pd.read_csv(file_path)
neighborhood_data.tail()


,Neighborhood,Lat,Lon
68,West Portal,37.74026,-122.463880
69,Western Addition,37.79298,-122.435790
70,Westwood Highlands,37.73470,-122.456854
71,Westwood Park,37.73415,-122.457000
72,Yerba Buena,37.79298,-122.396360


In [56]:
file_path = Path("/Users/domenicrhedrick/Desktop/Presentations/Pythonic Monopoly/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
sfo_data.tail()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2016,Telegraph Hill,903.049771,384242,4390
2016,Twin Peaks,970.085470,384242,4390
2016,Van Ness/ Civic Center,552.602567,384242,4390
2016,Visitacion Valley,328.319007,384242,4390
2016,Westwood Park,631.195426,384242,4390


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [60]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    sfo_data_year = sfo_data.groupby('year')
    housing_unit_yearly_mean = sfo_data_year['housing_units'].mean()
    housing_units_per_year = housing_unit_yearly_mean.hvplot.bar(ylim = (360000,390000))
    
    return housing_units_per_year

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    sfo_data_year = sfo_data.groupby('year')
    sfo_sqfoot_rent = sfo_data_year['sale_price_sqr_foot', 'gross_rent'].mean()
    avg_gross_rent = sfo_sqfoot_rent['gross_rent'].hvplot()

    return avg_gross_rent


def average_sales_price():
    """Average Sales Price Per Year."""
    
    sfo_data_year = sfo_data.groupby('year')
    sfo_sqfoot_rent = sfo_data_year['sale_price_sqr_foot', 'gross_rent'].mean()
    average_sales_price = sfo_sqfoot_rent['sale_price_sqr_foot'].hvplot()

    return average_sales_price



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    avg_neighborhood_year = sfo_data.groupby(['year', 'neighborhood']).mean()
    avg_price_neighborhood = avg_neighborhood_year['sale_price_sqr_foot'].hvplot('year', groupby='neighborhood', xlabel="Year", ylabel="Price Per Square Foot")

    return avg_price_neighborhood



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    avg_cost_neighborhood = sfo_data.groupby('neighborhood').mean()
    top_ten = avg_cost_neighborhood['sale_price_sqr_foot'].sort_values(ascending=False).head(10)
    ten_most_expensive_neighborhood = top_ten.hvplot.bar(rot=45, xlabel="Neighborhood", ylabel="Average Price Per Square Foot")

    return ten_most_expensive_neighborhood


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    avg_cost_neighborhood = sfo_data.groupby('neighborhood').mean()
    top_ten = avg_cost_neighborhood['sale_price_sqr_foot'].sort_values(ascending=False).head(10)
    top_ten.hvplot.bar(rot=45, xlabel="Neighborhood", ylabel="Average Price Per Square Foot")
     
    return most_expensive_neighborhoods_rent_sales

    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    avg_cost_neighborhood = sfo_data.groupby('neighborhood').mean()
    parallel_coordinates = px.parallel_coordinates(avg_cost_neighborhood, color='gross_rent') 
    
    return parallel_coordinates



def parallel_categories():
    """Parallel Categories Plot."""
    
    avg_cost_neighborhood = sfo_data.groupby('neighborhood').mean()
    
    parallel_categories = px.parallel_categories(
        avg_cost_neighborhood,
        dimensions=["sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "sale_price_sqr_foot": "Sale per Square Foot",
            "housing_units": "Number of Houses",
            "gross_rent": "Rent",
        },
        )

    return parallel_categories



def neighborhood_map():
    """Neighborhood Map."""

    avg_cost_neighborhood = sfo_data.groupby('neighborhood').mean()
    merged_sfo_data = pd.merge(neighborhood_data, avg_cost_neighborhood, left_on='Neighborhood', right_on='neighborhood')   
    neighborhood_map = px.scatter_mapbox(
        merged_sfo_data,
        lat= 'Lat',
        lon= 'Lon',
        size= 'housing_units',
        color= 'gross_rent',
        title= 'Rent Price Averages in SFO',
    )
    return neighborhood_map



def sunburst():
    """Sunburst Plot."""
    
    sunburst_chart = px.sunburst(
    sfo_data,
    names='neighborhood',
    parents='gross_rent',
    values='sale_price_sqr_foot',
    )

    return sunburst



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [63]:
# Create a Title for the Dashboard
geo_column = pn.Column(
    "## SanFrancisco Housing Data", housing_units_per_year(), neighborhood_map(), most_expensive_neighborhoods_rent_sales(),
    top_most_expensive_neighborhoods()
)

scatter_column = pn.Column(
    "## Housing Cost in San Francisco Per Year.",
    average_gross_rent(),
    average_sales_price(),
    average_price_by_neighborhood(),
    average_price_by_neighborhood()
    
)


# Create a tab layout for the dashboard

SanFran_Data = pn.Tabs(
    ('SanFran Housing Data', geo_column),
    ('SanFran Housing Cost', scatter_column)
    
)


# Create the dashboard
SanFran_Data

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/domenicrhedrick/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/jl/7bykfdr560q50k3f7sc9xpfc0000gn/T/ipykernel_8545/1200486138.py", line 2, in <cell line: 2>
    geo_column = pn.Column(
  File "/Users/domenicrhedrick/opt/anaconda3/lib/python3.9/site-packages/panel/layout/base.py", line 624, in __init__
    params['objects'] = [panel(pane) for pane in objects]
  File "/Users/domenicrhedrick/opt/anaconda3/lib/python3.9/site-packages/panel/layout/base.py", line 624, in <listcomp>
    params['objects'] = [panel(pane) for pane in objects]
  File "/Users/domenicrhedrick/opt/anaconda3/lib/python3.9/site-packages/panel/pane/base.py", line 54, in panel
    pane = PaneBase.get_pane_type(obj, **kwargs)(obj, **kwargs)
  File "/Users/domenicrhedrick/opt/anaconda3/lib/python3.9/site-packages/panel/interact.py", line 156, in __

## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
SanFran_Data.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()